In [1]:
# CAGED (Cadastro Geral de Empregados e Desempregados)
# name:         caged_xlsx_to_csv
# date:         29/03/2022 - 01/04/2022
# description:  caged xlsx file data processed, optimized and saved into csv file

import pandas as pd


In [2]:
# creates panda database with all file sheets               - dfb
# creates panda dataframe with specific sheets              - first_df
# creates /path/files names for getting and saving data     - file_in, file_out

file_in = '3-tabelas_MAR.xlsx'
#file_in = '3-tabelas_DEZ.xlsx'
file_out = 'caged.csv'

skiprows = 4  # [0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0]

dfb = pd.read_excel(file_in, sheet_name=None, skiprows=skiprows)
first_df = dfb['Tabela 5.1']
# df


In [3]:
# optimizes data mensal evolution by dates with Saldos

months_names_br = ('Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho',
                   'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro')
month_names_dec = ('01',      '02',        '03',    '04',    '05',
                   '06',    '07',    '08',     '09',       '10',      '11',       '12')

# prepare dict variable to the replacements
mbr2mdec = dict(zip(months_names_br, month_names_dec))

df2 = first_df.copy()

# selecting date cells only
df2 = df2[(df2['Mês'].str.slice(-4) >= '1900') &
          (df2['Mês'].str.slice(-4) <= '2099')]

# months name to months numbers replacements
df2['Mês'] = df2['Mês'].replace(mbr2mdec, regex=True)
df2['Mês'] = df2['Mês'].str.replace('/', '-')
df2['Mês'] = pd.to_datetime(
    df2['Mês'], format='%Y-%m-%d', infer_datetime_format=True)


def try_int(val):
    try:
        val = float(val)
        val = int(val)
        return True
    except:
        return False


filter = df2['Saldos'].apply(try_int)

df2.drop(df2[filter == False].index, inplace=True)
df2['Saldos'] = df2['Saldos'].astype(int)
first_out_df = df2[['Mês', 'Saldos']]
first_out_df


,Mês,Saldos
0,2020-01-01,111957
1,2020-02-01,217207
2,2020-03-01,-295055
3,2020-04-01,-981834
4,2020-05-01,-398443
5,2020-06-01,-54034
6,2020-07-01,108410
7,2020-08-01,214523
8,2020-09-01,299373
9,2020-10-01,366019


In [4]:
# creates panda dataframe with specific sheets  - second_df

second_df = pd.read_excel(file_in, sheet_name=11, skiprows=4, header=[0, 1])


In [5]:
# optimizes data mensal evolution by dates and regions with Saldos
months_names_br = ('Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho',
                   'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro')

regions_table = ('Região e UF', 'Unnamed: 1_level_1')
regions_names = ('Norte', 'Nordeste', 'Sudeste',
                 'Sul', 'Centro-Oeste', 'Não identificado')

# copying df for reutilizing
df2 = second_df.copy()

# function to create a dataframe columns list to use
def createColumnsList(param_df):
    columns_list = []
    column_names = param_df.columns.values.tolist()
    for column_name in column_names:
        # column_name has 2 parts (date, info_type)
        col_date, col_info = column_name
        month = col_date.split('/')[0]
        if month in months_names_br:
            if col_info.lower().find('saldos') != -1:
                columns_list.append(column_name)

    return(columns_list)


regions_list = createColumnsList(df2)


#df3 = df2.copy()
#for region in regions_list:
##for regions_list_index in range(0, 2):
#    print('-MES----------------------------------------------------')
#    print(region)
#    filter = df3[region].apply(try_int)
#    print('-FILTRO----------------------------------------------------')
#    print(filter)
#    df4 = df3[region]
#    df4.transpose
#    print('-COLUNA_SALDO----------------------------------------------')
#    print(df4[[region]])
#    print('-VALORES_INVALIDOS-----------------------------------------')
#    print( df4[filter == False] )
#    df3.drop(df4[filter == False].index, inplace=True, axis=1)
#    print('-COLUNA ARRUMADA-------------------------------------------')
#    print(df3[[region]])    
    
##print(df4[regions_list])

# selecting specifics regions
df3 = df2[df2[regions_table].isin(regions_names)]
second_out_df = df3[regions_list]
second_out_df = second_out_df.transpose()
second_out_df = df3[regions_list].astype(int)
second_out_df


,Janeiro/2020,Fevereiro/2020,Março/2020,Abril/2020,Maio/2020,Junho/2020,Julho/2020,Agosto/2020,Setembro/2020,Outubro/2020,...,Junho/2021,Julho/2021,Agosto/2021,Setembro/2021,Outubro/2021,Novembro/2021,Dezembro/2021,Janeiro/2022,Fevereiro/2022,Março/2022
,Saldos,Saldos,Saldos,Saldos,Saldos,Saldos,Saldos,Saldos,Saldos,Saldos,...,Saldos,Saldos,Saldos,Saldos,Saldos,Saldos,Saldos,Saldos,Saldos,Saldos
1,2818,10793,-7535,-32789,-12755,5518,15771,22841,21822,20811,...,24163,23541,22158,17538,9132,16709,-14252,2537,12727,2
9,-3397,2204,-70004,-153334,-64149,-10635,19988,55683,86278,64635,...,49110,52400,85190,94307,50737,54909,-17862,3734,28085,2
19,32192,107425,-159720,-505531,-211141,-47029,33731,86373,118147,173157,...,160355,152281,187941,142080,117617,169336,-146193,48447,162442,2
24,60161,70366,-40221,-225073,-91232,-6764,25929,36656,58123,86095,...,42305,40953,55732,47898,52318,51265,-82382,59653,82898,2
28,20170,26426,-17571,-65107,-19235,4740,12989,12962,14993,21157,...,37235,33932,31417,21906,17133,13338,-22731,33843,40930,2
33,13,-7,-4,0,69,136,2,8,10,164,...,1062,656,658,1055,1016,403,-90,2141,1425,2


In [6]:
# creates panda dataframe with specific sheets  - third_df

third_df = pd.read_excel(file_in, sheet_name=9, skiprows=4, header=[0, 1])


In [7]:
# optimizes data mensal evolution by dates and sectors with Saldos

gr_act_econ_table = (
    'Grupamento de Atividades Econômicas e Seção CNAE 2.0', 'Unnamed: 1_level_1')
sector_names = ('Agricultura, pecuária, produção florestal, pesca e aquicultura', 'Indústria geral', 'Construção',
                'Comércio; reparação de veículos automotores e motocicletas', 'Serviços', 'Não identificado***')

# copying df for reutilizing
df2 = third_df.copy()

sectors_list = createColumnsList(df2)

for sector in sectors_list:
    filter = df2[sector].apply(try_int)
    df2[sector].drop( df2[ filter == False ].index, inplace = True)

# selecting specifics sectors
df3 = df2[df2[gr_act_econ_table].isin(sector_names)]
third_out_df = df3[sectors_list]
third_out_df = df3[sectors_list].astype(int)
third_out_df = third_out_df.transpose()
third_out_df


,,1,2,7,8,9,27
Janeiro/2020,Saldos,16485,58261,35866,-51726,53071,0
Fevereiro/2020,Saldos,3666,39971,25990,9053,138528,-1
Março/2020,Saldos,-8575,-42788,-17646,-89225,-136821,0
Abril/2020,Saldos,-6833,-217738,-75300,-265612,-416351,0
Maio/2020,Saldos,14792,-107815,-23958,-106402,-175059,-1
Junho/2020,Saldos,37710,-8936,15568,-27560,-70816,0
Julho/2020,Saldos,22470,49444,40722,20805,-25031,0
Agosto/2020,Saldos,8025,90198,50520,43706,22074,0
Setembro/2020,Saldos,5321,111316,47379,66418,68939,0
Outubro/2020,Saldos,-4501,84138,35035,110621,140726,0


In [10]:
# creates csv file

first_columns_names_out = {'Mês': '', 'Saldos': 'total'}
second_columns_names_out = {1: 'reg1', 9: 'reg2',
                            19: 'reg3', 24: 'reg4', 28: 'reg5', 33: 'reg9'}
third_columns_names_out = {1: 'setA', 2: 'setE',
                           7: 'setF', 8: 'setG', 9: 'setU', 27: 'setZ'}

# remove index and columns
temp_first_out_df = first_out_df.reset_index(drop=True)
first_out_df.rename(columns=first_columns_names_out, inplace=True)

temp_second_out_df = second_out_df.reset_index(drop=True)
second_out_df.rename(columns=second_columns_names_out, inplace=True)

temp_third_out_df = third_out_df.reset_index(drop=True)
third_out_df.rename(columns=third_columns_names_out, inplace=True)

final_out_df = pd.concat(
    [temp_first_out_df, temp_second_out_df, temp_third_out_df], axis=1)
final_out_df.to_csv(file_out, index=False)
final_out_df


,,total,"(Janeiro/2020, Saldos)","(Fevereiro/2020, Saldos)","(Março/2020, Saldos)","(Abril/2020, Saldos)","(Maio/2020, Saldos)","(Junho/2020, Saldos)","(Julho/2020, Saldos)","(Agosto/2020, Saldos)",...,"(Dezembro/2021, Saldos)","(Janeiro/2022, Saldos)","(Fevereiro/2022, Saldos)","(Março/2022, Saldos)",setA,setE,setF,setG,setU,setZ
0,2020-01-01,111957,2818.0,10793.0,-7535.0,-32789.0,-12755.0,5518.0,15771.0,22841.0,...,-14252.0,2537.0,12727.0,2.0,16485,58261,35866,-51726,53071,0
1,2020-02-01,217207,-3397.0,2204.0,-70004.0,-153334.0,-64149.0,-10635.0,19988.0,55683.0,...,-17862.0,3734.0,28085.0,2.0,3666,39971,25990,9053,138528,-1
2,2020-03-01,-295055,32192.0,107425.0,-159720.0,-505531.0,-211141.0,-47029.0,33731.0,86373.0,...,-146193.0,48447.0,162442.0,2.0,-8575,-42788,-17646,-89225,-136821,0
3,2020-04-01,-981834,60161.0,70366.0,-40221.0,-225073.0,-91232.0,-6764.0,25929.0,36656.0,...,-82382.0,59653.0,82898.0,2.0,-6833,-217738,-75300,-265612,-416351,0
4,2020-05-01,-398443,20170.0,26426.0,-17571.0,-65107.0,-19235.0,4740.0,12989.0,12962.0,...,-22731.0,33843.0,40930.0,2.0,14792,-107815,-23958,-106402,-175059,-1
5,2020-06-01,-54034,13.0,-7.0,-4.0,0.0,69.0,136.0,2.0,8.0,...,-90.0,2141.0,1425.0,2.0,37710,-8936,15568,-27560,-70816,0
6,2020-07-01,108410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,22470,49444,40722,20805,-25031,0
7,2020-08-01,214523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,8025,90198,50520,43706,22074,0
8,2020-09-01,299373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5321,111316,47379,66418,68939,0
9,2020-10-01,366019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-4501,84138,35035,110621,140726,0


In [9]:
f_in = 'caged_2022_02.csv'
myf_in = 'caged.csv'

csv_df = pd.read_csv(f_in)
mycsv_df = pd.read_csv(myf_in)
csv_df.rename(columns={'Unnamed: 0': ''}, inplace=True)
mycsv_df.rename(columns={'Unnamed: 0': ''}, inplace=True)

#rdf = csv_df[list(mycsv_df.columns)]
# rdf
#mycsv_df == rdf
